In [2]:
import pandas as pd
import numpy as np

In [21]:
# dataset = pd.read_excel("./data/Анализ финансирования на 23.06.2022.xls", sheet_name="ОБ   ")
# dataset = pd.read_excel("./data/Анализ финансирования на 04.08.2022.xls", sheet_name="ОБ")
# dataset = pd.read_excel("./data/data2/Книга1.xls", sheet_name="ОБ   ")
dataset = pd.read_excel("./data/data3/Анализ финансирования на 08.09.2022.xls", sheet_name="ОБ   ")
dataset.head(20)

,АНАЛИЗ ФИНАНСИРОВАНИЯ за 2022 г.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,по состоянию на 08 сентября 2022 г.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,тыс.руб.,NaN
3,№№ п/п,NaN,Наименование статей расходов,Бюджетные ассигнования на 2022 год,Лимиты бюджетных обязательств на 2022 год,Кассовый план,NaN,Профинанси-ровано,% финансирования,NaN,NaN,Выставленные заявки на оплату расходов,NaN
4,NaN,NaN,NaN,NaN,NaN,Выделенный Облфином,Неиспользовано по состоянию \nна 08 сентября ...,NaN,от бюджетных ассигований на 2022 г.,от лимитов бюджетных обязательств на 2022 г.,от кассового плана,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,Итого из областного бюджета,16697495.92,16662235.61,11868234.0,606175.96,11131530.78,66.665869,66.806946,93.792647,130527.26,NaN
7,1,NaN,Обеспечение деятельности подведомственных учре...,5508678.7,5483149.7,3996696.6,348640.21,3526491.7,64.017016,64.315072,88.235162,121564.69,NaN
8,NaN,NaN,"Мероприятия, связанные с профилактикой и устра...",42555.07,35903.58,35903.57,25.53,35878.04,84.309672,99.928865,99.928893,0,NaN
9,2,NaN,Социальное обеспечение - всего\n в том ...,11078409.68,11070175.87,7783253.56,247663.16,7526627.96,67.939607,67.990139,96.702849,8962.44,NaN


In [7]:
def row_stringify(row):
	row = map(str, list(row.values))
	str_row = "".join(row)
	return str_row

def coords(dataset, name):
	row_index = 0
	for i in range(len(dataset)):
		row = dataset.iloc[i]
		row_str = row_stringify(row)
		if name in row_str:
			row_index = i
			break
	col_index = 0
	for j in range(len(dataset.columns)):
		col = dataset.iloc[row_index, j]
		if name in str(col):
			col_index = j
			break
	
	return np.array([row_index, col_index], dtype=np.int64)

In [24]:
start_coords_y, start_coords_x = coords(dataset, "Итого из областного бюджета")
start_coords_y_1, start_coords_x_1 = coords(dataset, "Бюджетные ассигнования на 2022 год")

In [25]:
dataset.iloc[start_coords_y, start_coords_x_1]

16697495.920000002

In [20]:
dataset.head(8)

,АНАЛИЗ ФИНАНСИРОВАНИЯ за 2022 г.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,по состоянию на 08 сентября 2022 г.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,тыс.руб.,NaN
3,№№ п/п,NaN,Наименование статей расходов,Бюджетные ассигнования на 2022 год,Лимиты бюджетных обязательств на 2022 год,Кассовый план,NaN,Профинанси-ровано,% финансирования,NaN,NaN,Выставленные заявки на оплату расходов,NaN
4,NaN,NaN,NaN,NaN,NaN,Выделенный Облфином,Неиспользовано по состоянию \nна 08 сентября ...,NaN,от бюджетных ассигований на 2022 г.,от лимитов бюджетных обязательств на 2022 г.,от кассового плана,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,Итого из областного бюджета,16697495.92,16662235.61,11868234.0,606175.96,11131530.78,66.665869,66.806946,93.792647,130527.26,NaN
7,1,NaN,Обеспечение деятельности подведомственных учре...,5508678.7,5483149.7,3996696.6,348640.21,3526491.7,64.017016,64.315072,88.235162,121564.69,NaN


In [23]:
dataset.iloc[start_coords_y, 3]

16697495.920000002

In [31]:
dataset.fillna(0, inplace=True)

In [ ]:
dataset.iloc[5]

In [17]:
new_dataset_columns = [
	'Дата',
 	'Наименование статей расходов',
 	'Бюджетные ассигнования на 2022 год ',
 	'Лимиты бюджетных обязательств на 2022 год',
 	'КП Выделенный Облфином',
 	'КП Неиспользовано по состоянию на 27 января 2022г.(не выставлено заявок)',
 	'Профинансировано',
 	'% от бюджетных ассигований на 2022 г.',
 	'% от лимитов бюджетных обязательств на 2022 г.',
 	'% от кассового плана',
 	'Выставленные заявки на оплату расходов ',
 	'Единица измерения'
]

flat_dataset = {name: [] for name in new_dataset_columns}
name = ""
subname = ""
empty_to_zero = lambda x: x if x != "" else 0

for i in range(start_coords_y, len(dataset)):
	# print(i)
	row = dataset.iloc[i]
	str_row = row_stringify(row=row)
	
	name = str(dataset.iloc[i, 2]).replace("\n", "")
	
	flat_dataset['Дата'].append("")
	flat_dataset['Наименование статей расходов'].append(name)
	flat_dataset['Бюджетные ассигнования на 2022 год '].append(empty_to_zero(dataset.iloc[i, start_coords_x_1]))
	flat_dataset['Лимиты бюджетных обязательств на 2022 год'].append(empty_to_zero(dataset.iloc[i, start_coords_x_1+1]))
	flat_dataset['КП Выделенный Облфином'].append(empty_to_zero(dataset.iloc[i, start_coords_x_1+2]))
	flat_dataset['КП Неиспользовано по состоянию на 27 января 2022г.(не выставлено заявок)'].append(empty_to_zero(dataset.iloc[i, start_coords_x_1+3]))
	flat_dataset['Профинансировано'].append(empty_to_zero(dataset.iloc[i, start_coords_x_1+4]))
	flat_dataset['% от бюджетных ассигований на 2022 г.'].append(empty_to_zero(dataset.iloc[i, start_coords_x_1+5]))
	flat_dataset['% от лимитов бюджетных обязательств на 2022 г.'].append(empty_to_zero(dataset.iloc[i, start_coords_x_1+6]))
	flat_dataset['% от кассового плана'].append(empty_to_zero(dataset.iloc[i, start_coords_x_1+7]))
	flat_dataset['Выставленные заявки на оплату расходов '].append(empty_to_zero(dataset.iloc[i, start_coords_x_1+8]))
	flat_dataset['Единица измерения'].append("тыс. руб")
		# print(dataset.iloc[i, 2])
	
	if "Аппарат комитета" in str_row:
		break

pd.DataFrame(data=flat_dataset)#.to_excel("./test.xlsx")


IndexError: single positional indexer is out-of-bounds

In [39]:
import importlib
importlib.reload(
	importlib.import_module("data_parser_7")
)
from data_parser_7 import DataParser_7
parser = DataParser_7()
parser.parse(
	input_data_path="./data/data2/Книга1.xls",
	date_creation="123",
	output_data_path="./test.xlsx"
)

In [34]:
# pd.read_excel("./data/data2/Книга1.xls", sheet_name="ОБ   ").head(20)

In [35]:
import pandas as pd
pd.read_excel("./test.xlsx")

,Дата,Наименование статей расходов,Бюджетные ассигнования на 2022 год,Лимиты бюджетных обязательств на 2022 год,КП Выделенный Облфином,КП Неиспользовано по состоянию на 27 января 2022г.(не выставлено заявок),Профинансировано,% от бюджетных ассигований на 2022 г.,% от лимитов бюджетных обязательств на 2022 г.,% от кассового плана,Выставленные заявки на оплату расходов,Единица измерения
0,123,0.00,16697544.98,16662284.71,11857998.55,1099779.42,10327887.33,61.852730,61.983621,87.096379,430331.80,тыс. руб
1,123,1.00,5520230.70,5494701.74,3957658.17,567963.86,3276313.33,59.351022,59.626773,82.784141,113380.98,тыс. руб
2,123,0.00,42545.11,35893.62,35893.61,15.57,35878.04,84.329409,99.956594,99.956622,0.00,тыс. руб
3,123,2.00,11067591.50,11059357.69,7812525.91,518032.21,6977550.88,63.044890,63.091828,89.312355,316942.82,тыс. руб
4,123,2.10,4218065.36,4218065.36,2839635.94,217882.24,2306391.56,54.678896,54.678896,81.221382,315362.14,тыс. руб
...,...,...,...,...,...,...,...,...,...,...,...,...
81,123,2.33,73186.40,69948.33,49501.81,2651.51,46850.30,64.015036,66.978440,94.643610,0.00,тыс. руб
82,123,2.34,16356.07,15588.17,11589.09,430.27,11158.82,68.224335,71.585183,96.287284,0.00,тыс. руб
83,123,2.35,4183.00,3666.63,2907.23,351.19,2556.04,61.105427,69.710879,87.920116,0.00,тыс. руб
84,123,2.36,19118.86,18149.45,16780.34,8390.03,8388.46,43.875315,46.218811,49.989810,1.85,тыс. руб
